In [ ]:
DEBUG = False
MODE = 'TRAIN'
#MODE = 'Inference'
MODEL_DIR = '../input/optiver-lgb-and-te-baseline'

In [ ]:
import numpy as np
import pandas as pd
import gc
import pathlib
from tqdm.auto import tqdm # Widget progress bar
import json
from multiprocessing import Pool, cpu_count
import time
import requests as re
from datetime import datetime
from dateutil.relativedelta import relativedelta, FR


import glob
import os
from sklearn import model_selection
import joblib
import lightgbm as lgb


import matplotlib.pyplot as plt
import matplotlib.style as style
from matplotlib_venn import venn2, venn3
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context('talk')
style.use('seaborn-colorblind')

import warnings
warnings.simplefilter('ignore')

pd.get_option('display.max_columns')


In [ ]:
class CFG:
    INPUT_DIR = '../input/optiver-realized-volatility-prediction'
    OUTPUT_DIR = './'

In [ ]:
def init_logger(log_file='train.log'):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter('%(message)s'))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter('%(message)s'))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = init_logger(log_file=f'{CFG.OUTPUT_DIR}/baseline.log')
logger.info(f'Start Logging...')

In [ ]:
train = pd.read_csv(os.path.join(CFG.INPUT_DIR, 'train.csv'))  # target, what you are trying to predict, first appears in train
test = pd.read_csv(os.path.join(CFG.INPUT_DIR, 'test.csv'))
ss = pd.read_csv(os.path.join(CFG.INPUT_DIR, 'sample_submission.csv'))
book = pd.read_parquet(os.path.join(CFG.INPUT_DIR, 'book_train.parquet'))
trade = pd.read_parquet(os.path.join(CFG.INPUT_DIR, 'trade_train.parquet'))

In [ ]:
train_book_stocks = os.listdir(os.path.join(CFG.INPUT_DIR, 'book_train.parquet'))

if DEBUG:
    logger.info('Debug mode: using 3 stocks only')
    train_book_stocks = train_book_stocks[:3]
    
logger.info('{:,} train book stocks: {}'.format(len(train_book_stocks), train_book_stocks))

In [ ]:
#load stock_id=0
def load_book(stock_id, data_type='train'):
    """
    load parquet book data for given stock_id
    """
    #the following line imports the parquet book train data and selects the specified stock_id to be stored in book_df
    book_df = pd.read_parquet(os.path.join(CFG.INPUT_DIR, f'book_{data_type}.parquet/stock_id={stock_id}')) 
    
    book_df['stock_id'] = stock_id
    book_df['stock_id'] = book_df['stock_id'].astype(np.int8)
    
    return book_df

In [ ]:
def load_trade(stock_id=0, data_type='train'):
    """
    load parquet trade data for given stock_id
    """
    #the following line imports the parquet trade train data and selects the specified stock_id to be stored in trade_df
    trade_df = pd.read_parquet(os.path.join(CFG.INPUT_DIR, f'trade_{data_type}.parquet/stock_id={stock_id}'))
    trade_df['stock_id'] = stock_id
    trade_df['stock_id'] = trade_df['stock_id'].astype(np.int8)
    
    return trade_df

In [ ]:
def fix_jsonerr(df):
    # fix json column error for lightgbm
    # isalnum returns true if all digits in string are alphanumeric
    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
    return df

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [ ]:
def realized_volatility(series_log_return):
    series_log_return = log_return(series_log_return)
    return np.sqrt(np.sum(series_log_return ** 2))

In [ ]:
def fe_row(book):
    #Feature engineering (just volatility) for each row
    
    #volatility
    for i in[1,2, ]:
        #wap
        book[f'book_wap{i}'] = (book[f'bid_price{i}'] * book[f'ask_size{i}'] + book[f'ask_price{i}'] * book[f'bid_size{i}']) / (book[f'bid_size{i}'] + book[f'ask_size{i}'])
            
    #mean wap
    book['book_wap_mean'] = (book['book_wap1'] + book['book_wap2']) / 2
    
    #wap diff
    book['book_wap_diff'] = book['book_wap1'] - book['book_wap2']
    
    #other orderbook features
    book['book_price_spread'] = (book['ask_price1'] - book['bid_price1']) / (book['ask_price1'] + book['bid_price1'])
    book['book_bid_spread'] = book['bid_price1'] - book['bid_price2']
    book['book_ask_spread'] = book['ask_price1'] - book['ask_price2']
    book['book_total_volume'] = book['ask_size1'] + book['ask_size2'] + book['bid_size1'] + book['bid_size2']
    book['book_volume_imbalance'] = (book['ask_size1'] + book['ask_size2']) - (book['bid_size1'] + book['bid_size2'])
    
    return book 

In [ ]:
def fe_agg(book_df):
    #feature engineering (aggregation by stock_id x time_id)
    
    # features
    book_feats = book_df.columns[book_df.columns.str.startswith('book_')].values.tolist()
    trade_feats = ['price', 'size', 'order_count', 'seconds_in_bucket']
    
    # agg trade features
    trade_df = book_df.groupby(['time_id', 'stock_id'])[trade_feats].agg(['sum', 'mean', 'std', 'max', 'min']).reset_index()
    
    # agg volatility features
    fe_df = book_df.groupby(['time_id', 'stock_id'])[book_feats].agg([realized_volatility]).reset_index()
    fe_df.columns = [" ".join(col).strip() for col in fe_df.columns.values]
    
    # merge
    fe_df = fe_df.merge(trade_df, how='left', on=['time_id', 'stock_id'])
    
    return fe_df

In [ ]:
# This is where the previous functions come together
def fe_all(book_df):
    """
    perform feature engineerings
    """
      
    # row-wise feature engineering, ADDS THE EXTRA FEATURES TO THE DATAFRAME
    book_df = fe_row(book_df)
    
    # feature engineering agg by stock_id x time_id, ADDS REALIZED VOLATILITY TO SOME OF THE FEATURES
    fe_df = fe_agg(book_df)
    
    return fe_df

In [ ]:
def book_fe_by_stock(stock_id=0):
    """
    load orderbook and trade data for the given stock_id and merge
    
    """
    # load data
    book_df = load_book(stock_id, 'train')
    trade_df = load_trade(stock_id, 'train')
    book_feats = book_df.columns.values.tolist()
    
    # merge
    book_df = book_df.merge(trade_df, how='outer', on=['time_id', 'seconds_in_bucket', 'stock_id'])
    
    # sort by time
    book_df = book_df.sort_values(by=['time_id', 'seconds_in_bucket'])
    
    # fillna for book_df
    book_df[book_feats] = book_df[book_feats].fillna(method='ffill')
    
    # feature engineering
    fe_df = fe_all(book_df)
    return fe_df

In [ ]:
def book_fe_by_stock_test(stock_id=0):
    """
    same function but for the test
    
    """
    # load data
    book_df = load_book(stock_id, 'test')
    trade_df = load_trade(stock_id, 'test')
    book_feats = book_df.columns.values.tolist()
    
    # merge
    book_df = book_df.merge(
        trade_df
        , how='outer'
        , on=['time_id', 'seconds_in_bucket', 'stock_id']
    )
    
    # sort by time
    book_df = book_df.sort_values(by=['time_id', 'seconds_in_bucket'])
    
    # fillna for book_df
    book_df[book_feats] = book_df[book_feats].fillna(method='ffill')
    
    # feature engineering
    fe_df = fe_all(book_df)
    return fe_df

In [ ]:
def book_fe_all(stock_ids, data_type='train'):
    # feature engineering with multithread processing
    
    # feature engineering agg by stock_id x time_id
    with Pool(cpu_count()) as p:
        if data_type == 'train':
            feature_dfs = list(tqdm(p.imap(book_fe_by_stock, stock_ids), total=len(stock_ids)))
        elif data_type == 'test':
            feature_dfs = list(tqdm(p.imap(book_fe_by_stock_test, stock_ids), total=len(stock_ids)))
            
    fe_df = pd.concat(feature_dfs)
    
    # feature engineering agg by stock_id
    vol_feats = [f for f in fe_df.columns if ('realized' in f) & ('wap' in f)]
    if data_type == 'train':
        # agg
        stock_df = fe_df.groupby('stock_id')[vol_feats].agg(['mean', 'std', 'max', 'min']).reset_index()
        
        # fix column names
        stock_df.columns = ['stock_id'] + [f'{f}_stock' for f in stock_df.columns.values.tolist()[1:]]
        stock_df = fix_jsonerr(stock_df)
        
    # feature engineering agg by time_id
    time_df = fe_df.groupby('time_id')[vol_feats].agg(['mean', 'std', 'max', 'min']).reset_index()
    time_df.columns = ['time_id'] + [f'{f}_time' for f in time_df.columns.values.tolist()[1:]]
    
    # merge
    fe_df = fe_df.merge(time_df, how='left', on='time_id')
    
    # make sure to fix json error for lightgbm
    fe_df = fix_jsonerr(fe_df)
    
    # out
    if data_type == 'train':
        return fe_df, stock_df
    elif data_type == 'test':
        return fe_df

In [ ]:
%%time

if MODE == 'TRAIN':
    # all book data feature engineering
    stock_ids = [int(i.split('=')[-1]) for i in train_book_stocks]
    book_df, stock_df = book_fe_all(stock_ids, data_type='train')
    
    assert book_df['stock_id'].nunique() > 2
    assert book_df['time_id'].nunique() > 2
    
    # save stock_df for the test
    stock_df.to_pickle('train_stock_df.pkl')
    logger.info('train stock df saved!')
    
    # merge 
    book_df = book_df.merge(stock_df, how='left', on='stock_id').merge(train, how='left', on=['stock_id', 'time_id']).replace([np.inf, -np.inf], np.nan).fillna(method='ffill')
    
    # make row_id
    book_df['row_id'] = book_df['stock_id'].astype(str) + '-' + book_df['time_id'].astype(str)
    
    print(book_df.shape)
    book_df.head()

In [ ]:
book_df.head()

In [ ]:
# test
test_book_stocks = os.listdir(os.path.join(CFG.INPUT_DIR, 'book_test.parquet'))

logger.info('{:,} test book stocks: {}'.format(len(test_book_stocks), test_book_stocks))

# all book data feature engineering
test_stocks_ids = [int(i.split('=')[-1]) for i in test_book_stocks]
test_book_df = book_fe_all(test_stocks_ids, data_type='test')

# load stock_df, if inference
if MODE == 'INFERENCE':
    stock_df = pd.read_pickle(f'{MODEL_DIR}/train_stock_df.pkl')
    
# merge
test_book_df = test.merge(stock_df, how='left', on='stock_id').merge(test_book_df, how='left', on=['stock_id', 'time_id']).replace([np.inf, -np.inf], np.nan).fillna(method='ffill')

# make row_id
test_book_df['row_id'] = test_book_df['stock_id'].astype(str) + '-' + test_book_df['time_id'].astype(str)

print(test_book_df.shape)
test_book_df.head()

In [ ]:
target = 'target'
drops = [target, 'row_id', 'time_id']
features = [f for f in test_book_df.columns.values.tolist() if (f not in drops) & (test_book_df[f].isna().sum() == 0) & (book_df[f].isna().sum() == 0)]
cats = ['stock_id']

logger.info('{:,} features ({:,} categorical): {}'.format(len(features), len(cats), features))

In [ ]:
# evaluation metric
def RMSPEMetric(XGBoost=False):
    
    def RMSPE(yhat, dtrain, XGBoost = XGBoost):
        
        y = dtrain.get_label()
        elements = ((y - yhat) / y) ** 2
        if XGBoost:
            return 'RMSPE', float(np.sqrt(np.sum(elements) / len(y)))
        else:
            return 'RMSPE', float(np.sqrt(np.sum(elements) / len(y))), False
        
    return RMSPE

In [ ]:
def fit_model(params, X_train, y_train, X_test, features=features, cats=[], era='stock_id', fold_type='kfold', n_fold = 5, seed=42):
    """
    fit model with cv (cross validation)
    """
    
    models = []
    oof_df = X_train[['time_id', 'stock_id', target]].copy()
    oof_df['pred'] = np.nan
    y_preds = np.zeros((len(X_test),))
    
    if fold_type == 'stratifiedshuffle':
        cv = model_selecion.StratifiedShuffleSplit(n_splits=n_fold, random_state=seed)
        kf = cv.split(X_train, X_train[era])
    elif fold_type == 'kfold':
        cv = model_selection.KFold(n_splits=n_fold, shuffle=True, random_state=seed)
        kf = cv.split(X_train, y_train)
    
    fi_df = pd.DataFrame()
    fi_df['features'] = features
    fi_df['importance'] = 0
    
    for fold_id, (train_index, valid_index) in tqdm(enumerate(kf)):
        #split
        X_tr = X_train.loc[train_index, features]
        X_val = X_train.loc[valid_index, features]
        y_tr = y_train.loc[train_index]
        y_val = y_train.loc[valid_index]
        
        
        # model, note inverse weighting please
        train_set = lgb.Dataset(X_tr, y_tr, categorical_feature = cats, weight = 1/np.power(y_tr, 2))
        val_set = lgb.Dataset(X_val, y_val, categorical_feature = cats, weight = 1/np.power(y_val, 2))
        # model training
        model = lgb.train(params, train_set, valid_sets = [train_set, val_set], feval=RMSPEMetric(), verbose_eval=250)
        
        # feature importance
        fi_df[f'importance_fold{fold_id}'] = model.feature_importance(importance_type='gain')
        fi_df['importance'] += fi_df[f'importance_fold{fold_id}'].values
        
        # save model
        joblib.dump(model, f'model_fold{fold_id}.pkl')
        logger.debug('model saved!')
        
        # predict
        oof_df['pred'].iloc[valid_index] = model.predict(X_val)
        y_pred = model.predict(X_test[features])
        y_preds += y_pred / n_fold
        models.append(model)
        
    return oof_df, y_preds, models, fi_df

### Notes on hyperparamters: 
#### num_leaves, max_depth (between 3 and 12), min_data_in_leaf in hundreds or thousands for big datasets (specifies the minimum number of observations that fit the decision criteria in a leaf.)  n_estimators, learning_rate, max_bin (maybe), lambda_l1, lambda_l2, min_gain_to_split, bagging_fraction, feature_fraction, bagging_freq

# Hyperparameter Tuning

In [ ]:
import optuna

def objective(trial, features=features, X=book_df, y=book_df[target], cats=[]):
    param_grid = {"n_estimators": trial.suggest_categorical("n_estimators", [813]),
        "boosting": trial.suggest_categorical("boosting",['dart']),
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 511, 750),
        #"max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 617, 800),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        #"lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        #"lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        #"min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 0.75),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1,7]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.8, 0.95)
    }
    
    cv = model_selection.KFold(n_splits=2, shuffle=True, random_state=46) #n_splits is reduced to 2 for time saving
    #kf = cv.split(X_train, y_train)
    cv_scores = np.empty(2) # from optuna blog
    
    # splits the data into groups, uses one as validation, trains models and gets results then repeats
    for fold_id, (train_index, valid_index) in tqdm(enumerate(cv.split(X,y))): 
        #split
        X_tr = X.loc[train_index, features]
        X_val = X.loc[valid_index, features]
        y_tr = y.loc[train_index]
        y_val = y.loc[valid_index]
        
        
        # model, note inverse weighting please
        train_set = lgb.Dataset(X_tr, y_tr, categorical_feature = cats, weight = 1/np.power(y_tr, 2))
        val_set = lgb.Dataset(X_val, y_val, categorical_feature = cats, weight = 1/np.power(y_val, 2))
        # Note that train() will return a model from the best iteration.
        model = lgb.train(param_grid, train_set, valid_sets = [train_set, val_set], feval=RMSPEMetric(), verbose_eval=250)
        #model.fit(X_train=book_df,y_train = book_df[target],test_book_df,cats = cats)
        
        # predict
        y_pred = model.predict(X_val[features])
        y_true = y_val
        return (np.sqrt(np.mean(np.square((y_true - y_pred)/y_true))))
    
    

In [ ]:
#optimization_function = partial(objective, X=x, y=y)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

In [ ]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
def fit_model(params, X_train, y_train, X_test, features=features, cats=[], era='stock_id', fold_type='kfold', n_fold = 2, seed=42):
    """
    fit model with cv (cross validation)
    """
    
    models = []
    oof_df = X_train[['time_id', 'stock_id', target]].copy()
    oof_df['pred'] = np.nan
    y_preds = np.zeros((len(X_test),))
    
    if fold_type == 'stratifiedshuffle':
        cv = model_selecion.StratifiedShuffleSplit(n_splits=n_fold, random_state=seed)
        kf = cv.split(X_train, X_train[era])
    elif fold_type == 'kfold':
        cv = model_selection.KFold(n_splits=n_fold, shuffle=True, random_state=seed)
        kf = cv.split(X_train, y_train)
    
    fi_df = pd.DataFrame()
    fi_df['features'] = features
    fi_df['importance'] = 0
    
    for fold_id, (train_index, valid_index) in tqdm(enumerate(kf)):
        #split
        X_tr = X_train.loc[train_index, features]
        X_val = X_train.loc[valid_index, features]
        y_tr = y_train.loc[train_index]
        y_val = y_train.loc[valid_index]
        
        
        # model, note inverse weighting please
        train_set = lgb.Dataset(X_tr, y_tr, categorical_feature = cats, weight = 1/np.power(y_tr, 2))
        val_set = lgb.Dataset(X_val, y_val, categorical_feature = cats, weight = 1/np.power(y_val, 2))
        # model training
        model = lgb.train(params, train_set, valid_sets = [train_set, val_set], feval=RMSPEMetric(), verbose_eval=250)
        
        # feature importance
        fi_df[f'importance_fold{fold_id}'] = model.feature_importance(importance_type='gain')
        fi_df['importance'] += fi_df[f'importance_fold{fold_id}'].values
        
        # save model
        joblib.dump(model, f'model_fold{fold_id}.pkl')
        logger.debug('model saved!')
        
        # predict
        oof_df['pred'].iloc[valid_index] = model.predict(X_val)
        y_pred = model.predict(X_test[features])
        y_preds += y_pred / n_fold
        models.append(model)
        
    return oof_df, y_preds, models, fi_df

In [ ]:
# %%time

params = {
    'n_estimators' : 813,
    'objective': 'rmse',
    'boosting_type': 'dart',
    'max_depth': -1,
    'learning_rate': 0.1,
    'num_leaves': 511,
    'min_data_in_leaf': 617,
    'max_bin': 70,
    'subsample': 0.82,
    'subsample_freq': 7,
    'feature_fraction': 0.6,
    'lambda_l1': 0.5,
    'lambda_l2': 1,
    'min_gain_to_split': 0.009275035155177913,
    'seed': 46,
    'early_stopping_rounds': 75,
    'verbose': -1
}

if MODE == 'TRAIN':
    oof_df, y_preds, models, fi_df = fit_model(params,
                                              book_df,
                                              book_df[target],
                                              test_book_df,
                                              features=features,
                                              cats = cats,
                                              era = None,
                                              fold_type = 'kfold',
                                              n_fold = 2,
                                              seed = 46)

## Feature Importance and Feature Reduction

In [ ]:
Feature Importance
if MODE == 'TRAIN':
    fi_df = fi_df.sort_values(by='importance', ascending=False)
    fi_df.to_csv('feature_importance.csv', index=False)
    
    #fig, ax = plt.subplots(1, 1, figsize=(10, 40))
    #sns.barplot(x='importance', y='features', data=fi_df.iloc[:30], ax=ax)
    logger.info(fi_df[['features', 'importance']].iloc[:50].to_markdown())

In [ ]:
reduced_features = (fi_df['features'].loc[:35])
reduced_features = reduced_features.to_list()
reduced_features

In [ ]:
reduced_features = ['book_wap1_realized_volatility',
 'book_wap2_realized_volatility',
 'book_wap_mean_realized_volatility',
 'stock_id',
 '__book_wap2_realized_volatility____min___time',
 'book_bid_spread_realized_volatility',
 '__book_wap_mean_realized_volatility____mean___time',
 '__book_wap1_realized_volatility____min___time',
 'book_total_volume_realized_volatility',
 '__book_wap_mean_realized_volatility____min___time',
 '__book_wap2_realized_volatility____mean___time',
 '__price____std__',
 '__book_wap1_realized_volatility____mean___stock',
 '__book_wap1_realized_volatility____mean___time',
 '__book_wap_diff_realized_volatility____min___time',
 '__book_wap1_realized_volatility____max___time',
 '__book_wap_mean_realized_volatility____max___time',
 '__book_wap2_realized_volatility____max___time',
 '__price____sum__',
 '__seconds_in_bucket____sum__',
 '__seconds_in_bucket____mean__',
 '__order_count____sum__',
 'book_wap_diff_realized_volatility',
 '__order_count____std__',
 '__book_wap_mean_realized_volatility____mean___stock',
 'book_price_spread_realized_volatility',
 '__seconds_in_bucket____std__',
 '__book_wap1_realized_volatility____min___stock',
 '__price____min__',
 '__size____max__',
 '__order_count____mean__',
 '__price____max__',
 '__size____mean__',
 '__size____std__',
 '__size____min__',
 '__book_wap2_realized_volatility____mean___stock',
 '__price____mean__',
 '__order_count____max__']

In [ ]:
# %%time

params_1 = {
    'n_estimators' : 813,
    'objective': 'rmse',
    'boosting_type': 'dart',
    'max_depth': -1,
    'learning_rate': 0.1,
    'num_leaves': 511,
    'min_data_in_leaf': 617,
    'max_bin': 70,
    'subsample': 0.82,
    'subsample_freq': 7,
    'feature_fraction': 0.6,
    'lambda_l1': 0.5,
    'lambda_l2': 1,
    #'min_gain_to_split': 0.009275035155177913,
    'seed': 46,
    'early_stopping_rounds': 50,
    'verbose': -1
}

if MODE == 'TRAIN':
    oof_df, y_preds, models, fi_df = fit_model(params_1,
                                              book_df,
                                              book_df[target],
                                              test_book_df,
                                              features=reduced_features,
                                              cats = cats,
                                              era = None,
                                              fold_type = 'kfold',
                                              n_fold = 3,
                                              seed = 46)

In [ ]:
MODE == 'INFERENCE'
if MODE == 'INFERENCE':
    """
    used for inference kernel only
    """
    y_preds = np.zeros(len(test_book_df))
    files = glob.glob(f'{MODEL_DIR}/*model*.pkl')
    assert len(files) > 0
    for i, f in enumerate(files):
        model = joblib.load(f)
        y_preds += model.predict(test_book_df[features])
    y_preds /= (i+1)
    
test_book_df[target] = y_preds

In [ ]:
# test
test_book_df[target] = y_preds

# save the submit file
sub = test_book_df[['row_id', target]]
sub.to_csv('submission.csv',index = False)

logger.info('submitted!')
sub.head()